In [5]:
import cv2
import mediapipe as mp
import time
import math

class handDetector():
    def __init__(self, mode=False, maxHands=2, modelComplexity=1, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.modelComplexity = modelComplexity
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands, self.modelComplexity, self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils

    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        #print(results.multi_hand_landmarks)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)
        return img

    def findPosition(self, img, handNo=0, draw=True):
        xList = []
        yList = []
        bbox = []
        self.lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                #print(id, lm)
                h, w, c = img.shape
                cx, cy = int(lm.x*w), int(lm.y*h)
                xList.append(cx)
                yList.append(cy)
                #print(id, cx, cy)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255,0,255), cv2.FILLED)
            xmin, xmax = min(xList), max(xList)
            ymin, ymax = min(yList), max(yList)
            bbox = xmin, ymin, xmax, ymax
            bbox = list(bbox)
            a = xmax - xmin
            b = ymax - ymin
            if len(bbox) == 4:
                if a >b:
                    if bbox[0] -((a - b) //2 ) > 0:
                        razn = (a - b) //2 
                        bbox[3] = bbox[3] + razn
                        bbox[0] = bbox[0] - razn
                    
                else:
                    if bbox[1] -((b - a) //2  ) > 0:
                        razn = (b - a) //2 
                        bbox[2] = bbox[2] + razn
                        bbox[1] = bbox[1] - razn
            print(bbox)
                

            if draw:
                cv2.rectangle(img, (bbox[0]-20, bbox[1]-20), (bbox[2]+20, bbox[3]+20), (0, 255, 0), 2)
        return self.lmList, bbox

    def findDistance(self, p1, p2, img, draw=True):
        x1, y1 = self.lmList[p1][1], self.lmList[p1][2]
        x2, y2 = self.lmList[p2][1], self.lmList[p2][2]
        cx, cy = (x1+x2)//2, (y1+y2)//2

        if draw:
            cv2.circle(img, (x1,y1), 15, (255,0,255), cv2.FILLED)
            cv2.circle(img, (x2,y2), 15, (255,0,255), cv2.FILLED)
            cv2.line(img, (x1,y1), (x2,y2), (255,0,255), 3)
            cv2.circle(img, (cx,cy), 15, (255,0,255), cv2.FILLED)

        length = math.hypot(x2-x1, y2-y1)
        return length, img, [x1, y1, x2, y2, cx, cy]

    def fingersUp(self):
        fingers = []

        # Thumb
        if self.lmList[self.tipIds[0]][1] < self.lmList[self.tipIds[0]-1][1]:
            fingers.append(1)
        else:
            fingers.append(0)

        # 4 Fingers
        for id in range(1,5):
            if self.lmList[self.tipIds[id]][2] < self.lmList[self.tipIds[id]-2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
        return fingers

pTime = 0
cTime = 0
cap = cv2.VideoCapture(0)
detector = handDetector()
while True:
    success, img = cap.read()
    img = cv2.flip(img,1) # Mirror flip
    img = detector.findHands(img)
    lmList = detector.findPosition(img)
    if len(lmList) != 0:
        print(lmList[1])
    cTime = time.time()
    fps = 1. / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, str(int(fps)), (10,70), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,255), 3)
    cv2.imshow("Image", img)
    cv2.waitKey(1)
    if cv2.waitKey(20) == 27: # exit on ESC
        break

cv2.destroyWindow("Image")
cap.release()
cv2.waitKey(1)


[299, 226, 457, 466]
[299, 226, 457, 466]
[299, 222, 461, 473]
[299, 222, 461, 473]
[299, 216, 467, 478]
[299, 216, 467, 478]
[306, 227, 475, 482]
[306, 227, 475, 482]
[310, 251, 471, 488]
[310, 251, 471, 488]
[311, 256, 467, 485]
[311, 256, 467, 485]
[313, 263, 464, 486]
[313, 263, 464, 486]
[311, 258, 468, 490]
[311, 258, 468, 490]
[309, 256, 468, 493]
[309, 256, 468, 493]
[309, 257, 468, 497]
[309, 257, 468, 497]
[307, 252, 468, 500]
[307, 252, 468, 500]
[307, 253, 467, 498]
[307, 253, 467, 498]
[306, 250, 468, 502]
[306, 250, 468, 502]
[306, 250, 468, 501]
[306, 250, 468, 501]
[304, 249, 469, 502]
[304, 249, 469, 502]
[274, 274, 442, 497]
[274, 274, 442, 497]
[157, 242, 364, 509]
[157, 242, 364, 509]
[142, 236, 353, 507]
[142, 236, 353, 507]
[105, 246, 336, 518]
[105, 246, 336, 518]
[95, 248, 332, 521]
[95, 248, 332, 521]
[91, 226, 331, 498]
[91, 226, 331, 498]
[93, 212, 332, 483]
[93, 212, 332, 483]
[105, 153, 348, 459]
[105, 153, 348, 459]
[110, 110, 361, 438]
[110, 110, 361, 438

[180, 51, 353, 402]
[180, 51, 353, 402]
[184, 87, 330, 364]
[184, 87, 330, 364]
[190, 50, 356, 400]
[190, 50, 356, 400]
[189, 86, 328, 346]
[189, 86, 328, 346]
[193, 73, 337, 355]
[193, 73, 337, 355]
[197, 47, 362, 399]
[197, 47, 362, 399]
[197, 60, 343, 370]
[197, 60, 343, 370]
[204, 61, 363, 391]
[204, 61, 363, 391]
[204, 61, 364, 390]
[204, 61, 364, 390]
[209, 63, 368, 391]
[209, 63, 368, 391]
[205, 64, 370, 391]
[205, 64, 370, 391]
[205, 65, 371, 391]
[205, 65, 371, 391]
[201, 66, 376, 394]
[201, 66, 376, 394]
[199, 80, 388, 399]
[199, 80, 388, 399]
[196, 82, 390, 403]
[196, 82, 390, 403]
[183, 90, 392, 406]
[183, 90, 392, 406]
[176, 94, 392, 407]
[176, 94, 392, 407]
[172, 96, 394, 409]
[172, 96, 394, 409]
[171, 95, 400, 410]
[171, 95, 400, 410]
[172, 96, 401, 410]
[172, 96, 401, 410]
[158, 103, 402, 411]
[158, 103, 402, 411]
[135, 121, 395, 412]
[135, 121, 395, 412]
[113, 135, 390, 413]
[113, 135, 390, 413]
[107, 136, 394, 417]
[107, 136, 394, 417]
[101, 138, 397, 421]
[101, 138, 

[320, 188, 462, 454]
[320, 188, 462, 454]
[318, 194, 462, 454]
[318, 194, 462, 454]
[320, 213, 452, 455]
[320, 213, 452, 455]
[321, 218, 459, 458]
[321, 218, 459, 458]
[318, 226, 460, 457]
[318, 226, 460, 457]
[318, 221, 459, 456]
[318, 221, 459, 456]
[315, 225, 454, 458]
[315, 225, 454, 458]
[316, 230, 452, 458]
[316, 230, 452, 458]
[313, 233, 449, 459]
[313, 233, 449, 459]
[313, 234, 448, 458]
[313, 234, 448, 458]
[313, 237, 449, 460]
[313, 237, 449, 460]
[313, 237, 451, 462]
[313, 237, 451, 462]
[314, 239, 449, 461]
[314, 239, 449, 461]
[314, 242, 448, 461]
[314, 242, 448, 461]
[316, 241, 449, 460]
[316, 241, 449, 460]
[317, 238, 452, 459]
[317, 238, 452, 459]
[318, 238, 454, 459]
[318, 238, 454, 459]
[318, 227, 459, 459]
[318, 227, 459, 459]
[316, 223, 460, 460]
[316, 223, 460, 460]
[316, 220, 462, 461]
[316, 220, 462, 461]
[316, 221, 462, 460]
[316, 221, 462, 460]
[317, 220, 463, 460]
[317, 220, 463, 460]
[317, 231, 460, 460]
[317, 231, 460, 460]
[317, 238, 456, 460]
[317, 238, 45

[295, 193, 451, 454]
[295, 193, 451, 454]
[294, 194, 450, 456]
[294, 194, 450, 456]
[292, 197, 449, 456]
[292, 197, 449, 456]
[293, 198, 449, 457]
[293, 198, 449, 457]
[290, 198, 448, 460]
[290, 198, 448, 460]
[294, 200, 453, 462]
[294, 200, 453, 462]
[296, 196, 455, 462]
[296, 196, 455, 462]
[295, 193, 455, 462]
[295, 193, 455, 462]
[295, 191, 455, 463]
[295, 191, 455, 463]
[297, 195, 455, 464]
[297, 195, 455, 464]
[298, 192, 457, 463]
[298, 192, 457, 463]
[298, 192, 456, 463]
[298, 192, 456, 463]
[298, 193, 456, 464]
[298, 193, 456, 464]
[296, 194, 454, 463]
[296, 194, 454, 463]
[298, 192, 458, 463]
[298, 192, 458, 463]
[297, 190, 458, 463]
[297, 190, 458, 463]
[296, 182, 455, 464]
[296, 182, 455, 464]
[296, 204, 441, 443]
[296, 204, 441, 443]
[312, 214, 468, 461]
[312, 214, 468, 461]
[316, 217, 471, 460]
[316, 217, 471, 460]
[319, 247, 468, 464]
[319, 247, 468, 464]
[326, 244, 478, 464]
[326, 244, 478, 464]
[329, 244, 478, 462]
[329, 244, 478, 462]
[333, 242, 484, 466]
[333, 242, 48

[288, 284, 418, 467]
[288, 284, 418, 467]
[286, 267, 420, 464]
[286, 267, 420, 464]
[279, 268, 412, 464]
[279, 268, 412, 464]
[278, 269, 409, 461]
[278, 269, 409, 461]
[276, 282, 403, 461]
[276, 282, 403, 461]
[277, 284, 404, 456]
[277, 284, 404, 456]
[279, 284, 405, 450]
[279, 284, 405, 450]
[277, 283, 406, 456]
[277, 283, 406, 456]
[277, 276, 411, 458]
[277, 276, 411, 458]
[278, 276, 412, 460]
[278, 276, 412, 460]
[279, 275, 411, 462]
[279, 275, 411, 462]
[278, 274, 412, 463]
[278, 274, 412, 463]
[278, 276, 410, 463]
[278, 276, 410, 463]
[278, 274, 410, 464]
[278, 274, 410, 464]
[278, 275, 409, 461]
[278, 275, 409, 461]
[277, 267, 410, 461]
[277, 267, 410, 461]
[278, 266, 409, 460]
[278, 266, 409, 460]
[278, 266, 410, 461]
[278, 266, 410, 461]
[277, 263, 410, 461]
[277, 263, 410, 461]
[278, 261, 412, 463]
[278, 261, 412, 463]
[278, 259, 412, 462]
[278, 259, 412, 462]
[279, 257, 414, 463]
[279, 257, 414, 463]
[276, 257, 413, 463]
[276, 257, 413, 463]
[276, 259, 411, 461]
[276, 259, 41

...

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[220, 160, 346, 322]
[220, 160, 346, 322]
[223, 168, 349, 328]
[223, 168, 349, 328]
[235, 161, 339, 319]
[235, 161, 339, 319]
[248, 179, 346, 326]
[248, 179, 346, 326]
[257, 242, 328, 335]
[257, 242, 328, 335]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[201, 164, 314, 290]
[201, 164, 314, 290]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[318, 13, 408, 160]
[318, 13, 408, 160]
[308, 61, 399, 164]
[308, 61, 399, 164]
[293, 21, 395, 141]
[293, 21, 395, 141]
[293, 22, 416, 128]
[293, 22, 416, 128]
[]
[]
[]
[]


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
